# Chạy thử  https://github.com/neubig/anlp-code/blob/main/03-lm/nn-lm.py

In [1]:
!git clone https://github.com/neubig/anlp-code.git

Cloning into 'anlp-code'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 52 (delta 15), reused 47 (delta 11), pack-reused 0
Unpacking objects: 100% (52/52), done.


In [2]:
%cd /content/anlp-code/03-lm

/content/anlp-code/03-lm


In [3]:
!python nn-lm.py

10000
Starting training epoch 1 over 42068 sentences
0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
4975it [00:15, 321.99it/s]--finished 5000 sentences (word/sec=6598.95)
9990it [00:31, 313.89it/s]--finished 10000 sentences (word/sec=6618.67)
14992it [00:47, 320.94it/s]--finished 15000 sentences (word/sec=6641.62)
19975it [01:03, 305.92it/s]--finished 20000 sentences (word/sec=6658.31)
24999it [01:18, 308.27it/s]--finished 25000 sentences (word/sec=6673.34)
29977it [01:34, 318.09it/s]--finished 30000 sentences (word/sec=6693.58)
34992it [01:50, 323.21it/s]--finished 35000 sentences (word/sec=6702.83)
39988it [02:05, 323.45it/s]--finished 40000 sentences (word/sec=6707.75)
42068it [02:12, 317.99it/s]
iter 0: train loss/word=6.0560, ppl=426.6677 (word/sec=6708.28)
epoch 0: dev loss/word=5.8711, ppl=354.6218 (word/sec=3

# Bài tập thực hành 2

In [9]:
%cd /content/

/content


In [10]:
!git clone https://github.com/NguyenXuanVinh2000/NLP-for-DataScience.git

Cloning into 'NLP-for-DataScience'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 0), reused 18 (delta 0), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [11]:
import tensorflow
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time
import random
import os, sys

In [ ]:
class FNN_LM(nn.Module):
  def __init__(self, nwords, emb_size, hid_size, num_hist):
    super(FNN_LM, self).__init__()
    self.embedding = nn.Embedding(nwords, emb_size)
    self.fnn = nn.Sequential(
      nn.Linear(num_hist*emb_size, hid_size),
      nn.Tanh(),
      nn.Linear(hid_size, nwords)
    )

  def forward(self, words):
    emb = self.embedding(words)      # 3D Tensor of size [batch_size x num_hist x emb_size]
    feat = emb.view(emb.size(0), -1) # 2D Tensor of size [batch_size x (num_hist*emb_size)]
    logit = self.fnn(feat)           # 2D Tensor of size [batch_size x nwords]

    return logit

N = 2 # The length of the n-gram
EMB_SIZE = 128 # The size of the embedding
HID_SIZE = 128 # The size of the hidden layer

USE_CUDA = torch.cuda.is_available()

# Functions to read in the corpus
# NOTE: We are using data from the Penn Treebank, which is already converted
#       into an easy-to-use format with "<unk>" symbols. If we were using other
#       data we would have to do pre-processing and consider how to choose
#       unknown words, etc.
w2i = {}
S = w2i["<s>"] = 0
UNK = w2i["<unk>"] = 1
# def get_wid(w2i, x, add_vocab=True):
#   if x not in w2i:
#     if add_vocab:
#       w2i[x] = len(w2i)
#     else:
#       return UNK
#   return w2i[x]
count = {}
def get_wid(w2i, x, add_vocab=True):
  if x not in w2i:
    if add_vocab:
      w2i[x] = len(w2i) 
      count[x] = 1
    else:
      return UNK
  else:
    count[x] = count[x] + 1
  return w2i[x]

def read_dataset(filename, add_vocab):
  with open(filename, "r") as f:
    for line in f:
      yield [get_wid(w2i, x, add_vocab)  for x in line.strip().split(" ")]

# Read in the data
train = list(read_dataset("/content/NLP-for-DataScience/Practice2/data/VLSP2013_raw_train.txt", add_vocab=True))
dev = list(read_dataset("/content/NLP-for-DataScience/Practice2/data/VLSP2013_raw_dev.txt", add_vocab=False))
test = list(read_dataset("/content/NLP-for-DataScience/Practice2/data/VLSP2013_raw_test.txt", add_vocab=False))

##1. Số câu văn huấn luyện trong tập train, dev và test là bao nhiêu?

In [13]:
print("Số câu văn huấn luyện trong tập train: ", len(train))
print("Số câu văn huấn luyện trong tập dev: ",len(dev))
print("Số câu văn huấn luyện trong tập test: ",len(test))

Số câu văn huấn luyện trong tập train:  23906
Số câu văn huấn luyện trong tập dev:  2009
Số câu văn huấn luyện trong tập test:  3481


##2. Kích thước của tập từ vựng là bao nhiêu?

In [12]:
i2w = {v: k for k, v in w2i.items()}
nwords = len(w2i)
print(nwords)
for key in count:
  if count[key] == 1:
    # print(key)
    w2i[key] = 1

24562


In [14]:

# Initialize the model and the optimizer
model = FNN_LM(nwords=nwords, emb_size=EMB_SIZE, hid_size=HID_SIZE, num_hist=N)
if USE_CUDA:
  model = model.cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# convert a (nested) list of int into a pytorch Variable
def convert_to_variable(words):
  var = Variable(torch.LongTensor(words))
  if USE_CUDA:
    var = var.cuda()

  return var

# A function to calculate scores for one value
def calc_score_of_histories(words):
  # This will change from a list of histories, to a pytorch Variable whose data type is LongTensor
  words_var = convert_to_variable(words)
  logits = model(words_var)
  return logits

# Calculate the loss value for the entire sentence
def calc_sent_loss(sent):
  # The initial history is equal to end of sentence symbols
  hist = [S] * N
  # Step through the sentence, including the end of sentence token
  all_histories = []
  all_targets = []
  for next_word in sent + [S]:
    all_histories.append(list(hist))
    all_targets.append(next_word)
    hist = hist[1:] + [next_word]

  logits = calc_score_of_histories(all_histories)
  loss = nn.functional.cross_entropy(logits, convert_to_variable(all_targets), size_average=False)

  return loss

MAX_LEN = 100
# Generate a sentence
def generate_sent():
  hist = [S] * N
  sent = []
  while True:
    logits = calc_score_of_histories([hist])
    prob = nn.functional.softmax(logits, 1)
    multinom = prob.multinomial(1)
    next_word = multinom.data.item()
    if next_word == S or len(sent) == MAX_LEN:
      break
    sent.append(next_word)
    hist = hist[1:] + [next_word]
  return sent

last_dev = 1e20
best_dev = 1e20




##3. Giá trị Perplexity (dựa trên per-word log likelihood) khi cho mô hình dự đoán trên tập train, dev và test là bao nhiêu?

In [15]:
for epoch in range(5):
  # Perform training
  random.shuffle(train)
  # set the model to training mode
  model.train()
  train_words, train_loss = 0, 0.0
  start = time.time()
  print(f'Starting training epoch {epoch+1} over {len(train)} sentences')
  for sent_id, sent in tqdm(enumerate(train)):
    my_loss = calc_sent_loss(sent)
    train_loss += my_loss.data
    train_words += len(sent)
    optimizer.zero_grad()
    my_loss.backward()
    optimizer.step()
    if (sent_id+1) % 5000 == 0:
      print("--finished %r sentences (word/sec=%.2f)" % (sent_id+1, train_words/(time.time()-start)))
  print("iter %r: train loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (epoch, train_loss/train_words, math.exp(train_loss/train_words), train_words/(time.time()-start)))
  
  # Evaluate on dev set
  # set the model to evaluation mode
  model.eval()
  dev_words, dev_loss = 0, 0.0
  start = time.time()
  for sent_id, sent in enumerate(dev):
    my_loss = calc_sent_loss(sent)
    dev_loss += my_loss.data
    dev_words += len(sent)

  # Keep track of the development accuracy and reduce the learning rate if it got worse

  if last_dev < dev_loss:
    # optimizer.param_group['lr'] /= 2

    for param_group in optimizer.param_groups: 
      param_group['lr'] /= 2 
  last_dev = dev_loss
  
  # Keep track of the best development accuracy, and save the model only if it's the best one
  if best_dev > dev_loss:
    torch.save(model, "model.pt")
    best_dev = dev_loss
  
  # Save the model
  print("epoch %r: dev loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (epoch, dev_loss/dev_words, math.exp(dev_loss/dev_words), dev_words/(time.time()-start)))



  test_words, test_loss = 0, 0.0
  start = time.time()
  test_ppl = {}
  for sent_id, sent in enumerate(test):
    my_loss = calc_sent_loss(sent)
    test_loss += my_loss.data
    test_words += len(sent)
    test_ppl[sent_id] = math.exp(test_loss/test_words)
  print("epoch %r: test loss/word=%.4f, ppl=%.4f (word/sec=%.2f)" % (epoch, test_loss/test_words, math.exp(test_loss/test_words), test_words/(time.time()-start)))
  # Generate a few sentences
  for _ in range(5):
    sent = generate_sent()
    print(" ".join([i2w[x] for x in sent]))

Starting training epoch 1 over 23906 sentences


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
5024it [00:29, 167.19it/s]

--finished 5000 sentences (word/sec=3960.02)


10024it [00:59, 167.33it/s]

--finished 10000 sentences (word/sec=3961.64)


...

--finished 15000 sentences (word/sec=3976.50)


20033it [02:00, 167.21it/s]

--finished 20000 sentences (word/sec=3975.89)


23906it [02:23, 166.40it/s]


iter 4: train loss/word=5.5619, ppl=260.3146 (word/sec=3975.80)
epoch 4: dev loss/word=7.0642, ppl=1169.3901 (word/sec=24640.56)
epoch 4: test loss/word=7.1729, ppl=1303.6820 (word/sec=23921.02)
Anh chỉ Pháp rộng vẫn 20 , mọi hành_vi có phần may_mắn tai_nạn thật với gia_đình anh còn 5 . 100 mìn cho người chiến_thắng .
- Nhóm quần_hệ rừng chữ nhà_nước và .
Vấn_đề mới chỉ là thế cũng không tay cả nhau .
Nhiều tháng để con một cuộc chủ cá Hải không sống .
Đồng_chí sẽ được đây là trong thực_tiễn .


In [16]:
test_ppl_show = dict(sorted(test_ppl.items(), key=lambda item: item[1]))
test_ppl_show = list(test_ppl_show.keys())
test_ppl_show[-5:]

[14, 10, 11, 13, 12]

##4. Ghi nhận lại 5 câu văn mà mô hình ngôn ngữ cho giá trị per-word log-likelihood cao nhất trên tập test. Phân tích nhanh kết quả thu được.

In [17]:
show_5_top = test_ppl_show[-5:]
for i in show_5_top:
  print(" ".join([i2w[x] for x in test[i]]))

Theo thông_báo của công_ty gửi về , Thanh mất ngày 19 - 8 - 2005 , khi đang làm_việc trên tàu đánh_cá ở vùng_biển gần Argentina nhưng mãi đến hơn tháng sau cả nhà mới biết tin .
Trên tay chị , bé <unk> cũng khóc <unk> đòi bú nghe não ruột , nát gan .
" Một tuần nay , khi biết tin chồng chết biển , o ( cô ) Minh có thiết ăn_uống gì đâu , làm_gì mà còn sữa nuôi con_bé " - bà hàng_xóm tên Ngân sụt_sùi lau ngang dòng lệ .
Một buổi chiều cuối tháng chín , chị Minh như sụp_đổ khi nhận được hung tin chồng mình là <unk> ( 28 tuổi ) đã chết khi đi đánh_cá trên bờ biển Nam_Mỹ ( anh Thanh là thuyền_viên VN mới nhất tử_nạn ) .
Ngoài_trời mưa <unk> , cảnh_vật quanh nhà vắng_vẻ , <unk> càng làm tăng thêm nỗi buồn người <unk> .


##5. Ghi nhận lại 5 câu văn mà mô hình ngôn ngữ cho giá trị per-word log-likelihood thấp nhất trên tập test. Phân tích nhanh kết quả thu được.

In [18]:
show_5_bot = test_ppl_show[0:5]
for i in show_5_bot:
  print(" ".join([i2w[x] for x in test[i]]))

Chị Minh ôm đứa con_gái mới hơn hai tháng rưỡi tuổi nấc lên từng tiếng thảm_thiết khi kể lại cho chúng_tôi nghe về cái chết của chồng .
Bọn trẻ cũng nhìn các ảnh này quen rồi " .
Năm tháng sau anh mất , bọn chúng vẫn còn quá nhỏ , chỉ biết mặt bố qua mấy tấm ảnh này " .
" Mỗi lần nhìn lại những ảnh này , tôi vẫn không_thể tin anh đã chết .
Chỉ có năm tấm ảnh do những đồng_nghiệp , chủ tàu chụp : cảnh mọi người đang bó cái xác <unk> của anh , cảnh <unk> , cảnh đưa thi_hài anh vào lò thiêu ...
